
## Step 1 : Load Labled Data for training  (With urban quality score from survey reflecting sujective preference)

### load the labled data set into two data frames:
### df_Q - Dataframe for lables of subjective quality score that we try to predict
### df_F - Dataframe for objective feature values to inform the prediction.


Load the CSV with Subjective Scores (label/y/dependent variable) and Objective Feature Values (X feature space/ independent variables) 

In [1]:
### Read BerlinTraining Data, please swap the Berlin file with the local file of your collection
import pandas as pd
df= pd.read_csv("Berlin_Data_forTraining/Berlin_Continuous.csv",encoding='utf-8') #, engine='python''gb2312','utf-8'
print(df.shape,df.columns)
df.head(2)

(300, 46) Index(['ID', 'JPG_ID', 'Lat', 'Lng', 'Q1_Text', 'Q1_Type', 'Q2_Order',
       'Q3_Access', 'Q4_Aesth', 'Q5_Eco', 'Q6_Enclo', 'Q7_Richness',
       'Q8_Scale', 'wall', 'building', 'sky', 'tree', 'road', 'sidewalk',
       'grass', 'earth', 'signboard', 'plant', 'fence', 'streetlight',
       'mountain', 'railing', 'column', 'bridge', 'water', 'ashcan', 'awning',
       'windowpane', 'skyscraper', 'ct_person', 'ct_bicycle', 'ct_car',
       'ct_motorcycle', 'ct_truck', 'ct_traffic light', 'ct_bus', 'ct_bench',
       'ct_boat', 'ct_stop sign', 'ct_parking meter', 'ct_dining table'],
      dtype='object')


,ID,JPG_ID,Lat,Lng,Q1_Text,Q1_Type,Q2_Order,Q3_Access,Q4_Aesth,Q5_Eco,...,ct_car,ct_motorcycle,ct_truck,ct_traffic light,ct_bus,ct_bench,ct_boat,ct_stop sign,ct_parking meter,ct_dining table
0,0,0,52.511540,13.305022,Comm,2,0.778286,0.840392,0.599666,0.360809,...,36,1,1,0,0,0,0,0,0,0
1,1,1,52.417602,13.140238,Rural,5,0.480052,0.260063,0.461102,0.611198,...,2,0,0,0,0,0,0,0,0,0


### Locate the lists of subjective quality scores that we try to predict into df_Q

To seperate contents of dataframe 'df' between row number x_1 to x_2, and between colunm number y_1 to y_2, then store them in 'df_Q'

df_Q=df.iloc[x_1:x_2, y_1:y_2]


To seperate contents of dataframe 'df' between row number x_1 to x_2, and keep all the colunms, then store them in dataframe 'df_Q'

df_Q=df.iloc[x_1:x_2, :]


To seperate contents of dataframe 'df' between colunm number y_1 to y_2, and keep all the rows, then store them in 'df_Q'

df_Q=df.iloc[:, y_1:y_2]

In [2]:
df_Q=df.iloc[:, 5:13] # please swap the colunm number here to the score of subjective features that you are trying to predict
df_Q.head(2) #this line prints first two row of the data frame

,Q1_Type,Q2_Order,Q3_Access,Q4_Aesth,Q5_Eco,Q6_Enclo,Q7_Richness,Q8_Scale
0,2,0.778286,0.840392,0.599666,0.360809,0.578130,0.714419,0.684783
1,5,0.480052,0.260063,0.461102,0.611198,0.310256,0.130178,0.257246


### Locate the list of urban objective features that we want to use for prediction into df_F

In [3]:
df_F=df.iloc[:,13:34] # please swap the colunm number here to the score urban features to inform prediction
print(df_F.shape)
df_F.head(2) #this line prints first two row of the data frame

(300, 21)


,wall,building,sky,tree,road,sidewalk,grass,earth,signboard,plant,...,streetlight,mountain,railing,column,bridge,water,ashcan,awning,windowpane,skyscraper
0,0.002605,0.171141,0.189527,0.190051,0.350102,0.011617,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.000000,0.127801,0.392734,0.307738,0.000000,0.113199,0.046516,0.000727,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
### Divide data for training & testing on Question 2-8
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error


ImportError: DLL load failed while importing _arpack: The specified procedure could not be found.

In [7]:
### Divide data for training & testing on Question 2-8
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt
import pickle
df_result=pd.DataFrame()

for i in range(len(df_Q.columns))[0:]:
    tag='TrainedModels/'+str(i)
    print (df_Q.columns[i])
    
    X = df_F.values  
    Y = df_Q.iloc[:,i].values #Quality Score at Colunm i
    xTr, xTe, yTr, yTe = train_test_split(X, Y, test_size=0.20)
    yTr,yTe=(yTr.ravel(),yTe.ravel())
    print(xTr.shape,xTe.shape)
    print(type(yTr))

    #KNN
    from sklearn.neighbors import KNeighborsRegressor
    KNN=KNeighborsRegressor().fit(xTr, yTr) #n_neighbors=2
    yPr=KNN.predict(xTe)
    filename = tag+'_model_KNN.sav'
    pickle.dump(KNN, open(filename, 'wb'))
    
    r2=round(r2_score(yTe,yPr),2)
    rmse=round(np.sqrt(np.sum((yPr-yTe)**2)/len(yPr)),2)
    mae=round(np.sum(np.sqrt((yPr-yTe)**2))/len(yPr),2)
    print("R2:%s, RMSE:%s, MAE:%s | KNN"%(r2,rmse,mae),'\n') # SVM_Reg.get_params
    df_result.loc[i,'KNN_R2']=r2
    df_result.loc[i,'KNN_RMSE']=rmse
    df_result.loc[i,'KNN_MAE']=mae

    #SVM
    from sklearn.svm import SVR
    SVM_Reg = SVR().fit(xTr,yTr)
    yPr=SVM_Reg.predict(xTe)
    filename = tag+'_model_SVM.sav'
    pickle.dump(SVM_Reg, open(filename, 'wb'))

    r2=round(r2_score(yTe,yPr),2)
    rmse=round(np.sqrt(np.sum((yPr-yTe)**2)/len(yPr)),2)
    mae=round(np.sum(np.sqrt((yPr-yTe)**2))/len(yPr),2)
    print("R2:%s, RMSE:%s, MAE:%s | SVM"%(r2,rmse,mae),'\n') # SVM_Reg.get_params
    df_result.loc[i,'SVM_R2']=r2
    df_result.loc[i,'SVM_RMSE']=rmse
    df_result.loc[i,'SVM_MAE']=mae

    #Random Forest
    from sklearn.ensemble import RandomForestRegressor
    RandomForest = RandomForestRegressor().fit(xTr,yTr)
    yPr=RandomForest.predict(xTe)
    filename = tag+'_model_RandomForest.sav'
    pickle.dump(RandomForest, open(filename, 'wb'))
    
    r2=round(r2_score(yTe,yPr),2)
    rmse=round(np.sqrt(np.sum((yPr-yTe)**2)/len(yPr)),2)
    mae=round(np.sum(np.sqrt((yPr-yTe)**2))/len(yPr),2)
    print("R2:%s, RMSE:%s, MAE:%s | Random Forest"%(r2,rmse,mae),'\n')
    df_result.loc[i,'RF_R2']=r2
    df_result.loc[i,'RF_RMSE']=rmse
    df_result.loc[i,'RF_MAE']=mae

    #Decision Tree
    from sklearn.tree import DecisionTreeRegressor
    DecisionTree= DecisionTreeRegressor(criterion='absolute_error',max_depth=2,random_state=10).fit(xTr, yTr)
    yPr=DecisionTree.predict(xTe)
    filename = tag+'_model_DecisionForest.sav'
    pickle.dump(DecisionTree, open(filename, 'wb'))
    
    r2=round(r2_score(yTe,yPr),2)
    rmse=round(np.sqrt(np.sum((yPr-yTe)**2)/len(yPr)),2)
    mae=round(np.sum(np.sqrt((yPr-yTe)**2))/len(yPr),2)
    print("R2:%s, RMSE:%s, MAE:%s | Decision Tree"%(r2,rmse,mae),'\n')
    df_result.loc[i,'DTree_R2']=r2
    df_result.loc[i,'DTree_RMSE']=rmse
    df_result.loc[i,'DTree_MAE']=mae

    #Linear Regression
    from sklearn.linear_model import LinearRegression
    lireg = LinearRegression().fit(xTr, yTr)
    yPr=lireg.predict(xTe)
    filename = tag+'_model_LinearRegression.sav'
    pickle.dump(lireg, open(filename, 'wb'))
    
    r2=round(r2_score(yTe,yPr),2)
    rmse=round(np.sqrt(np.sum((yPr-yTe)**2)/len(yPr)),2)
    mae=round(np.sum(np.sqrt((yPr-yTe)**2))/len(yPr),2)
    print("R2:%s, RMSE:%s, MAE:%s | OLS"%(r2,rmse,mae),'\n')
    df_result.loc[i,'LR_R2']=r2
    df_result.loc[i,'LR_RMSE']=rmse
    df_result.loc[i,'LR_MAE']=mae

    #Gaussian Process
    from sklearn.gaussian_process import GaussianProcessRegressor
    from sklearn.gaussian_process.kernels import DotProduct,WhiteKernel
    kernel = DotProduct() + WhiteKernel()
    gpr = GaussianProcessRegressor(kernel=kernel,random_state=0).fit(xTr, yTr)
    gpr.score(xTr, yTr)
    yPr=gpr.predict(xTe, return_std=True)
    yPr=yPr[0]
    filename = tag+'_model_Gaussian_Process.sav'
    pickle.dump(gpr, open(filename, 'wb'))
    
    r2=round(r2_score(yTe,yPr),2)
    rmse=round(np.sqrt(np.sum((yPr-yTe)**2)/len(yPr)),2)
    mae=round(np.sum(np.sqrt((yPr-yTe)**2))/len(yPr),2)
    print("R2:%s, RMSE:%s, MAE:%s | Gaussian"%(r2,rmse,mae),'\n')
    df_result.loc[i,'Gaus_R2']=r2
    df_result.loc[i,'Gaus_RMSE']=rmse
    df_result.loc[i,'Gaus_MAE']=mae

    #Voting Selection
    from sklearn.ensemble import VotingRegressor
    from sklearn.linear_model import LinearRegression
    from sklearn.ensemble import RandomForestRegressor
    from sklearn.ensemble import VotingRegressor
    
    r1 = LinearRegression()
    r2 = RandomForestRegressor(n_estimators=10, random_state=1)
    er = VotingRegressor([('lr', r1), ('rf', r2)]).fit(xTr, yTr)
    yPr=er.predict(xTe)
    filename = tag+'_model_Voting_Selection.sav'
    pickle.dump(er, open(filename, 'wb'))
    
    r2=round(r2_score(yTe,yPr),2)
    rmse=round(np.sqrt(np.sum((yPr-yTe)**2)/len(yPr)),2)
    mae=round(np.sum(np.sqrt((yPr-yTe)**2))/len(yPr),2)
    print("R2:%s, RMSE:%s, MAE:%s | Voting Selection"%(r2,rmse,mae),'\n')
    df_result.loc[i,'VtSel_R2']=r2
    df_result.loc[i,'VtSel_RMSE']=rmse
    df_result.loc[i,'VtSel_MAE']=mae

    # GradientBoosting Regression; To use this experimental feature, we need to explicitly ask for it:
    from sklearn.experimental import enable_hist_gradient_boosting  # noqa
    from sklearn.ensemble import HistGradientBoostingRegressor
    from sklearn.datasets import load_boston
    est = HistGradientBoostingRegressor().fit(xTr,yTr)
    yPr=est.predict(xTe)
    #print(est.score(xTr, yTr))
    filename = tag+'_model_GradientBoosting_Regression.sav'
    pickle.dump(est, open(filename, 'wb'))
    
    r2=round(r2_score(yTe,yPr),2)
    rmse=round(np.sqrt(np.sum((yPr-yTe)**2)/len(yPr)),2)
    mae=round(np.sum(np.sqrt((yPr-yTe)**2))/len(yPr),2)
    print("R2:%s, RMSE:%s, MAE:%s |GradientBoosting"%(r2,rmse,mae),'\n')
    df_result.loc[i,'GB_R2']=r2
    df_result.loc[i,'GB_RMSE']=rmse
    df_result.loc[i,'GB_MAE']=mae

    # ADA Boost Regression
    from sklearn.ensemble import AdaBoostRegressor
    adaBst = AdaBoostRegressor(random_state=0, n_estimators=100).fit(xTr,yTr)
    #adaBst.feature_importances_
    yPr=adaBst.predict(xTe)
    filename = tag+'_model_ADABoosting_Regression.sav'
    pickle.dump(adaBst, open(filename, 'wb'))
    
    r2=round(r2_score(yTe,yPr),2)
    rmse=round(np.sqrt(np.sum((yPr-yTe)**2)/len(yPr)),2)
    mae=round(np.sum(np.sqrt((yPr-yTe)**2))/len(yPr),2)
    print("R2:%s, RMSE:%s, MAE:%s |ADA Boost"%(r2,rmse,mae),'\n')
    df_result.loc[i,'AB_R2']=r2
    df_result.loc[i,'AB_RMSE']=rmse
    df_result.loc[i,'AB_MAE']=mae

    # BagingRegressor
    from sklearn.svm import SVR
    from sklearn.ensemble import BaggingRegressor
    regr = BaggingRegressor(base_estimator=SVR(),n_estimators=10, random_state=0).fit(xTr, yTr)
    yPr=regr.predict(xTe)
    filename = tag+'_model_BagingRegressor.sav'
    pickle.dump(regr, open(filename, 'wb'))
    
    r2=round(r2_score(yTe,yPr),2)
    rmse=round(np.sqrt(np.sum((yPr-yTe)**2)/len(yPr)),2)
    mae=round(np.sum(np.sqrt((yPr-yTe)**2))/len(yPr),2)
    print("R2:%s, RMSE:%s, MAE:%s |Bagging Regression"%(r2,rmse,mae),'\n')
    df_result.loc[i,'BR_R2']=r2
    df_result.loc[i,'BR_RMSE']=rmse
    df_result.loc[i,'BR_MAE']=mae

ImportError: DLL load failed while importing _arpack: The specified procedure could not be found.

### Save the performance chart of different models for different quality predictions in an excel

In [ ]:
df_result.to_csv('TrainedModels/Example_ModelSelection_Continuous.csv', index=False, encoding='utf-8')
df_result